In [1]:
import pickle
import numpy as np
import pandas as pd
from pprint import pprint
from time import time
import logging
import sys
import tensorflow as tf
from os import path
from os import listdir
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
#import tensorflow_hub as hub
from tensorflow.keras.callbacks import TensorBoard
from azureml.core import Dataset, Workspace
#import tensorflow_addons as tfa

state_gov_nm = 'CA'
year_to_process = '2018'
data_file_location = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/quigley-gpu-instance/code/QuigleyData'
#vector_size = [128]
#cluster_size = [50]
vector_size = [32, 64, 96, 128, 160]#, 192, 256]
cluster_size = [10, 30, 50]#, 70, 100, 150, 200]
_accuracy_log = path.join(data_file_location, state_gov_nm + 'accuracy_log.txt')
#state_gov_nm = sys.argv[1]
#year_to_process = sys.argv[2]
#data_file_location = sys.argv[3]


In [2]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
#labels are derived from the "passed" column
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('passed')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
#if 1==1:
with open(_accuracy_log, 'w') as lg:
    for v in vector_size:
        for c in cluster_size:
            _dataset = path.join(data_file_location, state_gov_nm + str(v) + str(c) + '_dataset.df')
            with open(_dataset, 'rb') as f:
                leg_data = pickle.load(f)

            train_set = leg_data[leg_data['year']<int(year_to_process)]
            test_set = leg_data[leg_data['year']==int(year_to_process)]

            test = test_set
            #create a balanced train/test split

            data_pass = leg_data[leg_data['passed']==1]
            data_fail = leg_data[leg_data['passed']==0]

            if len(data_pass) > len(data_fail):
                data_pass  = data_pass.iloc[:len(data_fail)]
            else:
                data_fail = data_fail.iloc[:len(data_pass)]

            print(len(data_pass))
            print(len(data_fail))

            #combine the balanced sets and shuiffle
            balanced_train = pd.concat([data_pass, data_fail])
            balanced_train = balanced_train.sample(frac=1)
            balanced_train = balanced_train.sample(frac=1)

            #pull out a validation set
            train, val = train_test_split(balanced_train, test_size=.25)

            print(len(train), 'train examples')
            print(len(val), 'validation examples')
            print(len(test), 'test examples')

            test = test.sample(frac=1)
            test = test.sample(frac=1)
            #print(test.head())
            test = test.drop_duplicates(subset='bill_id')
            feature_columns = []

            numericfeaturelist = [
                                    'sponsor_vec', #change to -1 to 1
                                    'partisan_vector'
                                  ]

            for dv in range(1, v+1):
                numericfeaturelist.append('dv' + str(dv))

            sponsors_cols = [col for col in leg_data.columns if 'sponsors_' in col]

            numericfeaturelist.extend(sponsors_cols)

            #numeric cols
            for header in numericfeaturelist:
                feature_columns.append(feature_column.numeric_column(header))

            # indicator cols
            primary_party = feature_column.categorical_column_with_vocabulary_list('primary_party', leg_data['primary_party'].unique())
            primary_party_onehot = feature_column.indicator_column(primary_party)
            feature_columns.append(primary_party_onehot)

            time_t = feature_column.categorical_column_with_vocabulary_list('time_t', leg_data['time_t'].unique())
            time_t_onehot = feature_column.indicator_column(time_t)
            feature_columns.append(time_t_onehot)

            primary_sponsor = feature_column.categorical_column_with_vocabulary_list('primary_name', leg_data['primary_name'].unique())
            primary_sponsor_onehot = feature_column.indicator_column(primary_sponsor)
            feature_columns.append(primary_sponsor_onehot)

            bill_category = feature_column.categorical_column_with_vocabulary_list('category', leg_data['category'].unique())
            bill_category_onehot = feature_column.indicator_column(bill_category)
            feature_columns.append(bill_category_onehot)

            # embedding cols
            event_list = feature_column.categorical_column_with_vocabulary_list('cmltv_event', leg_data['cmltv_event'].unique())
            event_list_onehot = feature_column.indicator_column(event_list)
            event_list_embed = feature_column.embedding_column(event_list, dimension=16)

            #feature_columns.append(event_list_embed)
            feature_columns.append(event_list_onehot)
            #primary_embedding = feature_column.embedding_column(primary_sponsor, dimension=8)

            feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


            batch_size = 1024
            train_ds = df_to_dataset(train, shuffle = True, batch_size=batch_size)
            val_ds = df_to_dataset(val, shuffle=True, batch_size=batch_size)
            test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

            model = tf.keras.Sequential([
                  feature_layer,
                  layers.Dense(1000, activation='relu'),
                  layers.Dropout(.5),
                  layers.Dense(250, activation='relu'),
                  layers.Dropout(.25),
                  layers.Dense(1000, activation='relu'),
                  layers.Dropout(.5),
                  layers.Dense(1, activation='sigmoid')
                ])

            model.compile(optimizer='adam',
                          loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                          #loss=tf.nn.sigmoid_cross_entropy_with_logits(),
                          metrics=['accuracy'
                                   , tf.keras.metrics.Precision()
                                   , tf.keras.metrics.Recall()
                                  # , tfa.metrics.F1Score()
                                  ])

            model.fit(train_ds,
                      #validation_data = test_ds,
                      validation_data=val_ds,
                      epochs=25#, callbacks = [tensorboard]
                     )

            loss, accuracy, precision, recall = model.evaluate(test_ds)
            print("Vector Length + Cluster Size:",v,c,"Accuracy:", accuracy, "Precision:", precision, "Recall:", recall, file=lg)
        

152687
152687
229030 train examples
76344 validation examples
84036 test examples


WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:4267: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/feature_column/feature_column_v2.py:4322: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Train for 224 steps, validate for 75 steps
Epoch 1/25
224/224 [==============================] - 198s 885ms/step - loss: 0.6101 - accuracy: 0.7622 - precision: 0.8197 - recall: 0.6720 - val_loss: 0.5874 - val_accuracy: 0.8009 - val_precision: 0.8922 - val_recall: 0.6854
Epoch 2/25
224/224 [==============================] - 174s 777ms/step - loss: 0.5783 - accuracy: 0.8370 - precision: 0.8727 - recall: 0.7888 - val_loss: 0.5679 - val_accuracy: 0.8529 - val_precision: 0.9071 - val_recall: 0.7870
Epoch 3/25
224/224 [==============================] - 189s 842ms/step - loss: 0.5649 - accuracy: 0.8657 - precision: 0.8987 - recall: 0.8240 - val_loss: 0.5567 - val_accuracy: 0.8774 - val_precision: 0.9286 - val_recall: 0.8183
Epoch 4/25
224/224 [==============================] - 174s 778ms/step - loss: 0.5552 - accuracy: 0.8873 - precision: 0.9150 - recall: 0.8537 - val_loss: 0.5464 - val_accuracy: 0.9083 - val_precision: 0.9223 - val_recall: 0.8921
Epoch 5/25
224/224 [=========================